# Local travel support chatbot with memory

Create a customer support chatbot for a travel agency with memory


In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="embaas/sentence-transformers-multilingual-e5-base")

In [ ]:
loader = DirectoryLoader("faq/", glob="**/*txt")
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
vectorStore = Chroma.from_documents(texts, embeddings)

In [ ]:
template = """
### Instruction: You're a travelling support agent that is talking to a customer.

Use only the chat history and the following information
{context}
to answer in a helpful manner to the question.  If you don't know the answer -
say that you don't know. Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()

In [ ]:
prompt = PromptTemplate(
    input_variables=[ "context", "question", "chat_history"], template=template
)

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    input_key="question",
    output_key="output_text",
    return_messages=False,
)

In [ ]:
llm = LlamaCpp(
    model_path="models/llama-2-7b-chat.Q4_K_M.gguf",
    n_gpu_layers=1,
    max_tokens=512,
    callback_manager=callback_manager,
    verbose=False,
)

In [ ]:
chain = load_qa_chain(
    llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=False
)

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        break
    docs = vectorStore.similarity_search(user_input)
    answer = chain({"input_documents": docs, "question": user_input})
    print(answer)